In [33]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np

In [45]:
df = pd.read_csv("METABRIC_RNA_Mutation.csv").drop(columns=["patient_id"])
df

/tmp/ipykernel_50511/1861818716.py:1: DtypeWarning: Columns (678,688,690,692) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("METABRIC_RNA_Mutation.csv").drop(columns=["patient_id"])


,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,er_status,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
2,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
3,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
4,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,43.10,BREAST CONSERVING,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0,LumA,4.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1900,42.88,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1901,62.90,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1902,61.16,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,4.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0


In [46]:
df["overall_survival"].value_counts()

overall_survival
0    1103
1     801
Name: count, dtype: int64

In [47]:
original_length = df.shape[0]
df = df.dropna()
print(f"Original Length: {original_length}\nAfter null drop length: {df.shape[0]}")

Original Length: 1904
After null drop length: 1092


In [48]:
df["overall_survival"].value_counts()

overall_survival
0    608
1    484
Name: count, dtype: int64

In [49]:
columns_to_drop = []
for col in df.columns:
    if str(df[col].dtype) == "object":
        columns_to_drop.append(col)
columns_to_drop

['type_of_breast_surgery',
 'cancer_type',
 'cancer_type_detailed',
 'cellularity',
 'pam50_+_claudin-low_subtype',
 'er_status_measured_by_ihc',
 'er_status',
 'her2_status_measured_by_snp6',
 'her2_status',
 'tumor_other_histologic_subtype',
 'inferred_menopausal_state',
 'integrative_cluster',
 'primary_tumor_laterality',
 'oncotree_code',
 'pr_status',
 '3-gene_classifier_subtype',
 'death_from_cancer',
 'pik3ca_mut',
 'tp53_mut',
 'muc16_mut',
 'ahnak2_mut',
 'kmt2c_mut',
 'syne1_mut',
 'gata3_mut',
 'map3k1_mut',
 'ahnak_mut',
 'dnah11_mut',
 'cdh1_mut',
 'dnah2_mut',
 'kmt2d_mut',
 'ush2a_mut',
 'ryr2_mut',
 'dnah5_mut',
 'herc2_mut',
 'pde4dip_mut',
 'akap9_mut',
 'tg_mut',
 'birc6_mut',
 'utrn_mut',
 'tbx3_mut',
 'col6a3_mut',
 'arid1a_mut',
 'lama2_mut',
 'notch1_mut',
 'cbfb_mut',
 'ncor2_mut',
 'col12a1_mut',
 'col22a1_mut',
 'pten_mut',
 'akt1_mut',
 'atr_mut',
 'thada_mut',
 'ncor1_mut',
 'stab2_mut',
 'myh9_mut',
 'runx1_mut',
 'nf1_mut',
 'map2k4_mut',
 'ros1_mut',
 'la

In [50]:
columns_count = df.shape[1]
df = df.drop(columns=columns_to_drop)
print(f"Original Cols: {columns_count}\nAfter null drop length: {df.shape[1]}")

Original Cols: 692
After null drop length: 502


In [51]:
survival_1 = df[df["overall_survival"] == 1]
survival_0 =resample(df[df["overall_survival"] == 0], n_samples=survival_1.shape[0])
df = pd.concat([survival_1, survival_0])
df

,age_at_diagnosis,chemotherapy,cohort,neoplasm_histologic_grade,hormone_therapy,lymph_nodes_examined_positive,mutation_count,nottingham_prognostic_index,overall_survival_months,overall_survival,...,srd5a1,srd5a2,srd5a3,st7,star,tnk2,tulp4,ugt2b15,ugt2b17,ugt2b7
1,43.19,0,1.0,3.0,1,0.0,2.0,4.020,84.633333,1,...,-0.4412,0.4534,0.4068,0.7634,0.0231,0.9121,-0.9538,-0.2264,0.5398,-0.8920
19,45.43,1,1.0,3.0,1,0.0,5.0,4.046,140.866667,1,...,1.8932,-0.5688,-1.3892,0.9588,-0.0417,-0.6983,-1.1513,-0.3737,-0.1071,-0.7333
22,61.49,0,1.0,2.0,1,1.0,3.0,4.032,157.433333,1,...,-1.2288,0.4227,-0.1570,0.6778,1.6169,-1.6327,0.9158,5.0569,-0.7728,2.8982
31,46.89,0,1.0,2.0,1,0.0,3.0,3.068,148.033333,1,...,0.7693,-1.0047,-0.3767,0.7846,0.2334,-1.4285,-1.4207,3.7802,-1.1134,2.7724
33,49.87,1,1.0,1.0,1,5.0,4.0,4.140,85.333333,1,...,-0.2678,1.9758,1.1248,0.0801,0.0935,-0.1223,-0.0312,-0.4591,-0.3754,-0.7703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,57.98,0,3.0,3.0,1,9.0,5.0,6.030,111.566667,0,...,0.1539,0.4315,-0.0844,-1.1365,-0.4443,0.6097,2.0752,-0.1821,-0.2287,-0.8529
1246,69.59,0,3.0,2.0,1,0.0,4.0,3.060,102.066667,0,...,-0.3987,-1.2536,0.2386,-1.2347,-0.6900,1.8736,-0.0026,-0.2210,-0.7745,-0.7978
1454,75.28,0,3.0,2.0,1,0.0,9.0,3.050,71.633333,0,...,-1.1956,-0.1145,-0.9339,1.7156,0.0250,0.4448,0.1905,0.5219,0.8057,2.0751
248,74.58,0,1.0,3.0,1,13.0,6.0,6.086,61.100000,0,...,-0.2372,0.4518,-0.6113,-0.5981,-0.9868,-0.7854,0.1252,0.3379,-1.5421,0.0634


In [52]:
df["overall_survival"].value_counts()

overall_survival
1    484
0    484
Name: count, dtype: int64

In [53]:
response = df["overall_survival"]
data = df.drop(columns=["overall_survival"])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(data, response, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [55]:
X_train.to_csv("data_separated/train_data.csv", index=False)
X_val.to_csv("data_separated/val_data.csv", index=False)
X_test.to_csv("data_separated/test_data.csv", index=False)
y_train.to_csv("data_separated/y_train.csv", index=False)
y_val.to_csv("data_separated/y_val.csv", index=False)
y_test.to_csv("data_separated/y_test.csv", index=False)